In [ ]:
from langchain import SerpAPIWrapper
from langchain import OpenAI
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory, ReadOnlySharedMemory
from langchain.chains import LLMChain
from langchain.agents import (
    Tool,
    AgentExecutor,
    ZeroShotAgent
)

## Adding Share Memory to an Agents and its Tools

We are going to create a custom Agent. The agent has access to a conversation memory, search tool, and a summarization tool. And, the summarization tool also needs access to the conversation memory.

In [ ]:
## Setting the Summarization Chain

template = """This is a conversation between a human and a bot:

{chat_history}

Write a summary of the conversation for {input}:
"""

prompt = PromptTemplate(
    input_variables = ["input", "chat_history"],
    template = template
)

memory = ConversationBufferMemory(memory_key="chat_history")

readonlymemory = ReadOnlySharedMemory(memory=memory)

summry_chain = LLMChain(
    llm = OpenAI(openai_api_key=open("openai_api.txt").read()),
    prompt = prompt,
    verbose = True,
    memory = readonlymemory, # use the read-only memory to prevent the tool from modifying the memory
)

In [ ]:
## Setting Tools

search = SerpAPIWrapper(serpapi_api_key = open("serpapi_api.txt").read())

tools = [
    Tool(
        name = "Search",
        func=search.run,
        description="useful for when you need to answer questions about current events"
    ),
    Tool(
        name = "Summary",
        func=summry_chain.run,
        description="useful for when you summarize a conversation. The input to this tool should be a string, representing who will read this summary."
    )
]

In [ ]:
## Creating the Agent's Prompt

prefix = """Have a conversation with a human, answering the following questions as best you can. You have access to the following tools:"""
suffix = """Begin!"

{chat_history}
Question: {input}
{agent_scratchpad}"""

prompt = ZeroShotAgent.create_prompt(
    tools = tools,
    prefix = prefix,
    suffix = suffix,
    input_variables = ["input", "chat_history", "agent_scratchpad"]
)

In [ ]:
## Setting the Agent Executor

llm_chain = LLMChain(
    llm = OpenAI(temperature=0, openai_api_key=open("openai_api.txt").read()),
    prompt = prompt
)

agent = ZeroShotAgent(
    llm_chain = llm_chain,
    tools = tools,
    verbose = True
)

agent_chain = AgentExecutor.from_agent_and_tools(
    agent = agent,
    tools = tools,
    verbose = True,
    memory = memory
)

In [ ]:
agent_chain.run(input="What is ChatGPT?")

In [ ]:
agent_chain.run(input="Who developed it?")

In [ ]:
agent_chain.run(input="Thanks. Summarize the conversation, for my daughter 5 years old.")

In [ ]:
print(agent_chain.memory.buffer)